In [50]:
import pandas as pd

In [51]:
INPUT = 'data/raw/test.csv'
OUTPUT = INPUT.replace('raw', 'processed')

In [52]:
df = pd.read_csv(INPUT)

In [53]:
df_id = df['Id']
df.drop('Id', axis=1, inplace=True)

In [54]:
# Preencher valores nulos numéricos de df
num_cols = df.select_dtypes(include=['float64', 'int64']).columns

# Se uma coluna numérica tiver mais que 80% de valores nulos, ela é removida
for column in num_cols:
    if df[column].isnull().sum() > 0.8 * df.shape[0]:
        df.drop(column, axis=1, inplace=True)

num_cols = df.select_dtypes(include=['float64', 'int64']).columns

# Remover coluna Id salvando em um novo dataframe
for col in num_cols:
    df[col].fillna(df[col].mean(), inplace=True)

/tmp/ipykernel_23706/3212242508.py:13: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(df[col].mean(), inplace=True)


In [55]:
df.columns

Index(['MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street', 'Alley',
       'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope',
       'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle',
       'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'RoofStyle',
       'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'MasVnrArea',
       'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond',
       'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1', 'BsmtFinType2',
       'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating', 'HeatingQC',
       'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF',
       'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath',
       'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual', 'TotRmsAbvGrd',
       'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType', 'GarageYrBlt',
       'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual', 'GarageCond',
       'PavedDrive', 'Wo

In [56]:
categorical_cols = df.select_dtypes(include=['object']).columns
# Quais colunas do df tem valores nulos?
df[categorical_cols].isnull().sum()


# Colunas para usar fillna com a string 'NA':
cols_fillna_na = ['FireplaceQu', 'PoolQC', 'Fence', 'MiscFeature', 'Alley']

# Colunas para usar fillna com a string 'None':
cols_fillna_none = ['MasVnrType']

# Colunas para usar fillna com a moda (todas as outras):
cols_fillna_mode = [x for x in categorical_cols if x not in cols_fillna_na and x not in cols_fillna_none]

In [57]:
for col in cols_fillna_na:
    df[col].fillna('NA', inplace=True)

for col in cols_fillna_none:
    df[col].fillna('None', inplace=True)

for col in cols_fillna_mode:
    df[col].fillna(df[col].mode()[0], inplace=True)

/tmp/ipykernel_23706/4126170580.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna('NA', inplace=True)
/tmp/ipykernel_23706/4126170580.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.me

In [58]:
# Dropando colunas do df
df.drop("Alley", axis = 1, inplace=True)
df.drop("Utilities", axis = 1, inplace=True)
df.drop("BsmtHalfBath", axis = 1, inplace=True)

categorical_cols = categorical_cols.drop("Alley")
categorical_cols = categorical_cols.drop("Utilities")

In [59]:
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder(drop='if_binary', sparse_output=False)
data = ohe.fit_transform(df[categorical_cols])
features = ohe.get_feature_names_out(categorical_cols)
data = pd.DataFrame(data, columns=features)
df.drop(categorical_cols, axis=1, inplace=True)
df = pd.concat([df, data], axis=1)

In [60]:
# Concatena Id de volta
df = pd.concat([df_id, df], axis=1)

In [61]:
# Assert se há algum valor nulo
assert df.isnull().sum().sum() == 0

In [62]:
df

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,1461,20,80.0,11622,5,6,1961,1961,0.0,468.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1462,20,81.0,14267,6,6,1958,1958,108.0,923.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1463,60,74.0,13830,5,5,1997,1998,0.0,791.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1464,60,78.0,9978,6,6,1998,1998,20.0,602.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1465,120,43.0,5005,8,5,1992,1992,0.0,263.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,2915,160,21.0,1936,4,7,1970,1970,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1455,2916,160,21.0,1894,4,5,1970,1970,0.0,252.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
1456,2917,20,160.0,20000,5,7,1960,1996,0.0,1224.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
1457,2918,85,62.0,10441,5,5,1992,1992,0.0,337.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


In [63]:
df.to_csv(OUTPUT, index=False)